In [2]:
!pip install torch

In [3]:
!pip install torchvision

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#We are using the same MNIST dataset with drawn numbers 0-10

transform = transforms.Compose([
    transforms.ToTensor(), # Convert the images to pyTorch tensors
    transforms.Normalize((0.5,), (0.5,)) #Normalize the images 
])

# Load the datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [6]:
#Creating the Neural Network, the input nodes on the next layer should be the same as the output nodes of the previous layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        #First Layer 
        self.fc1 = nn.Linear(28 * 28, 128) #28x28 Image with 128 layer nodes
        #Second Layer
        self.fc2 = nn.Linear(128, 64) #128 input nodes and 64 output nodes
        #Final Layer
        self.fc3 = nn.Linear(64, 10) #64 input nodes and 10 output nodes for numbers 1-9 (this should be the amount of classes)

    def forward(self, x):
        x = x.view(-1, 28 * 28) #This makes the 2D matrix images into a 1D matrix image
        x = torch.relu(self.fc1(x)) #Activate the first layer
        x = torch.relu(self.fc2(x)) #Activate the second layer
        x = self.fc3(x) #Final layer
        return x

model = SimpleNN()


In [7]:
#Difining the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
num_epochs = 5

for epoch in range(num_epochs):
    for images, labels in train_loader:
        #Pass through the model forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        #Fix the model by using optimization and back-propogation
        optimizer.zero_grad() #Clear gradients
        loss.backward() #Back propogation
        optimizer.step() #Update weights from optimization and back propogation
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.1598
Epoch [2/5], Loss: 0.0894
Epoch [3/5], Loss: 0.0185
Epoch [4/5], Loss: 0.1359
Epoch [5/5], Loss: 0.0263


In [9]:
#Model Output
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

: 

In [ ]:
def show_model_output(model, test_loader):
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1) #Get the predicted class

            #Display the images and predictions
            for i in range(5): #First 5 images
                image = images[i].numpy().squeeze()
                label = labels[i].item()
                pred = predicted[i].item()

                plt.subplot(1, 5, i + 1)
                plt.imshow(image, cmap='gray')
                plt.title(f'true: {label}\nPred: {pred}')
                plt.axis('off')
            
            plt.show()
            break

show_model_output(model, test_loader)